In [ ]:

#Definition of the image embedder class

from torchvision import transforms as ts
import torchvision.models as models
class ImageEmbedder:
    def __init__(self):
        self.normalize = ts.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
        # https://pytorch.org/vision/0.8/models.html
        self.model = models.squeezenet1_0(pretrained=True, progress=False)

    def embed(self, image_file_name):
        image = Image.open(image_file_name).convert("RGB")
        image = ts.Resize(256)(image)
        image = ts.CenterCrop(224)(image)
        tensor = ts.ToTensor()(image)
        tensor = self.normalize(tensor).reshape(1, 3, 224, 224)
        vector = self.model(tensor).cpu().detach().numpy().flatten()
        return vector
    
image_embedder = ImageEmbedder()

In [ ]:

#Download the dataset

from datasets import load_dataset
dataset = load_dataset("chittaranjankhatua/car_damage_pub")

#Writes dataset to file
for i in range(len(dataset['train']["image"])):
    image = dataset['train']["image"][i]
    image_name = str('car_damage/')+str(i)+str('.jpg')
    image.save(image_name)

In [ ]:
#Connect to MDB
import bson
import pymongo
import pymongoarrow
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

#Insert your uri here
uri = "mongodb+srv://<username>:<password>clustername.n0kts.mongodb.net/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


In [ ]:
import os

db = client['claim_resolution']
coll = db['car_damage_photos']

image_folder = 'car_damage'

image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(image_folder, image_file)

    # Read the image from file as binary data
    with open(image_path, 'rb') as image_file_obj:
        image_data = image_file_obj.read()
    
    img_embedding = image_embedder.embed(image_path).tolist()
    
    image_document = {
        'filename': image_file,
        'data': image_data,
        'embedding': img_embedding
    }

    coll.insert_one(image_document)
    print(f"Inserted: {image_file}")

print("All images inserted into MongoDB.")



In [ ]:
import os

db = client['claim_resolution']
coll = db['car_damage_photos']

# Add query image to folder, in this example it has been named "test.jpg"
query_image = 'car_damage/test.jpg'

image_folder = 'car_damage'

query_embedding = image_embedder.embed(query_image).tolist()

documents = coll.aggregate([
            {
            "$search": {
            "index": "default",
            "knnBeta": {
            "vector": query_embedding,
            "path": "embedding",
            "k": 5
            }
            }
            }
            ])

documents = list(documents)  

similar_images_list = []

for i in range (5):
    image_file = documents[i]['filename']
    image_path = os.path.join(image_folder, image_file)
    similar_images_list.append(image_path)